In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
from trl import DPOConfig, DPOTrainer

In [ ]:
from transformers import LEDForConditionalGeneration, LEDTokenizer

In [ ]:
model_name = "allenai/led-large-16384-arxiv"

In [ ]:
tokenizer = LEDTokenizer.from_pretrained(model_name)
model = LEDForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

In [ ]:
ft_model_name = model_name.split('/')[1].replace("Instruct", "DPO")

In [ ]:
training_args = DPOConfig(
    output_dir=ft_model_name,
    logging_steps=25,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="epoch",
    eval_strategy="epoch",
    eval_steps=1,
)

In [ ]:
# set gpu as cuda device


NameError: name 'torch' is not defined

In [ ]:
df = pd.read_csv('manually_picked_articles.csv')

In [ ]:
training_dataset = df.drop(["article_index"], axis=1)[0:8]

In [ ]:
# if output_b_preferred, add selected_output_b to choosen and selected_output_a to rejected
# if output_b_preferred is 0, then reverse

training_dataset = training_dataset.rename(columns={'article': 'prompt'})
training_dataset

,prompt,selected_output_a,selected_output_b,output_b_preferred
0,Published as a conference paper at ICLR 2024TO...,large language models (LLMs ) have recently d...,large language models (LLMs ) have recently d...,0
1,Published as a conference paper at ICLR 2024MA...,we present a specifically pre-trained languag...,we present a specifically pre-trained languag...,1
2,Under review as a conference paper at ICLR 202...,instruction fine-tuning has recently emerged ...,instruction fine-tuning has recently emerged ...,1
3,Published as a conference paper at ICLR 2024TO...,despite the advancements of open-source large...,despite the advancements of open-source large...,1
4,Published as a conference paper at ICLR 2024IN...,instruction-tuned language models have demons...,instruction-tuned language models have demons...,1
5,7.3Figure 5: Ablation study of our method desi...,"in this conference paper , we present a new me...",* abstract *,0
6,Published as a conference paper at ICLR 2024LA...,multiple choice questions (MCQs ) serve as a ...,multiple choice questions (MCQs ) serve as a ...,0
7,Published as a conference paper at ICLR 2024BE...,we present the first com-prehensive study on ...,we present the first com-prehensive study on ...,1


In [ ]:
# Create 'chosen' and 'rejected' columns based on 'output_b_preferred'
training_dataset['chosen'] = training_dataset.apply(
    lambda row: row['selected_output_b'] if row['output_b_preferred'] else row['selected_output_a'],
    axis=1
)
training_dataset['rejected'] = training_dataset.apply(
    lambda row: row['selected_output_a'] if row['output_b_preferred']  else row['selected_output_b'],
    axis=1
)

In [ ]:
training_dataset.drop(['selected_output_a',	'selected_output_b',	'output_b_preferred'], axis=1, inplace=True)

In [ ]:
val_dataset = df.drop(["article_index"], axis=1)[9:12]

val_dataset = val_dataset.rename(columns={'article': 'prompt'})
val_dataset

,prompt,selected_output_a,selected_output_b,output_b_preferred
9,SST-20.8680.895Trec0.3110.653Mpqa0.6900.807Dis...,we present a prompt engineering and inference...,we present a prompt engineering method that c...,1
10,CommonsenseUnderstandingIs the model accuratel...,metacognitionDid the model respond with aware...,metacognitionDid the model respond with aware...,0


In [ ]:
# Create 'chosen' and 'rejected' columns based on 'output_b_preferred'
val_dataset['chosen'] = val_dataset.apply(
    lambda row: row['selected_output_b'] if row['output_b_preferred'] else row['selected_output_a'],
    axis=1
)
val_dataset['rejected'] = val_dataset.apply(
    lambda row: row['selected_output_a'] if row['output_b_preferred']  else row['selected_output_b'],
    axis=1
)

In [ ]:
val_dataset.drop(['selected_output_a',	'selected_output_b',	'output_b_preferred'], axis=1, inplace=True)

In [ ]:
### convert dataframes to dataset
from datasets import Dataset

training_dataset = Dataset.from_pandas(training_dataset)
val_dataset = Dataset.from_pandas(val_dataset)

AttributeError: 'Dataset' object has no attribute 'columns'

In [ ]:
training_dataset.save_to_disk("training_dataset")
val_dataset.save_to_disk("val_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/8 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
!pip install datasets

In [ ]:
!mkdir trained_dataset
!mkdir val_dataset

In [ ]:
from datasets import load_from_disk

In [ ]:
training_dataset = load_from_disk("trained_dataset")
val_dataset = load_from_disk("val_dataset")

In [ ]:
trainer = DPOTrainer(
    model=model,
    args=training_args,
    processing_class=tokenizer,
    train_dataset=training_dataset,
    eval_dataset=val_dataset,
)

Extracting prompt in train dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Extracting prompt in eval dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
!pwd

/content


In [ ]:
!mkdir trained_dataset

mkdir: cannot create directory ‘trained_dataset’: File exists


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: prathyushaguduru28 (prathyushaguduru28-univeristy-of-texas-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Input ids are automatically padded from 512 to 1024 to be a multiple of `config.attention_window`: 1024


Epoch,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/chosen,Logps/rejected,Logits/chosen,Logits/rejected
1,No log,0.615243,0.040112,-0.132816,0.500000,0.172928,-1521.968994,-669.408325,-2.221047,-2.272917
2,No log,0.620125,0.277194,0.087897,0.500000,0.189297,-1519.598145,-667.201172,-2.219081,-2.270617
3,No log,0.611245,0.359674,0.153473,0.500000,0.206201,-1518.773315,-666.545410,-2.218566,-2.270481


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['led.encoder.embed_tokens.weight', 'led.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=24, training_loss=3.3273448944091797, metrics={'train_runtime': 157.3681, 'train_samples_per_second': 0.153, 'train_steps_per_second': 0.153, 'total_flos': 0.0, 'train_loss': 3.3273448944091797, 'epoch': 3.0})

In [ ]:
model.save_pretrained("model.pt")

In [ ]:
import pandas as pd

df = pd.read_csv('articl_2.csv')

In [ ]:
ft_model = trainer.model

In [ ]:
article = df['article'][0]

In [ ]:
encoding = tokenizer(article, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

In [ ]:
!CUDA_LAUNCH_BLOCKING=1

In [ ]:
# Now, run inference with error handling (optional but recommended)
try:
    outputs = trainer.model(**encoding)
    # Further processing of outputs...
except RuntimeError as e:
    if "CUDA error: device-side assert triggered" in str(e):
        print("CUDA error detected! Check input data and tokenization process.")
        # Add more detailed debugging/logging here
        # For example, examine input_ids, attention_mask for unexpected values.
    else:
        raise e # Re-raise other exceptions


CUDA error detected! Check input data and tokenization process.


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_id = "Prathyusha101/LED_rlhf_ft"

In [ ]:
trainer.push_to_hub(model_id)

training_args.bin:   0%|          | 0.00/6.26k [00:00<?, ?B/s]

events.out.tfevents.1744515936.f4286ac5b5e2.204.0:   0%|          | 0.00/9.22k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Prathyusha101/led-large-16384-arxiv/commit/725ed37428962089144343b5a33ad7de86f6609f', commit_message='Prathyusha101/LED_rlhf_ft', commit_description='', oid='725ed37428962089144343b5a33ad7de86f6609f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Prathyusha101/led-large-16384-arxiv', endpoint='https://huggingface.co', repo_type='model', repo_id='Prathyusha101/led-large-16384-arxiv'), pr_revision=None, pr_num=None)